# Experiment (3)_TFIDF_(1,3)Max_features

Configurations

In [1]:
import mlflow
import os

# Set DagsHub credentials (you'll need to set these environment variables)
os.environ["MLFLOW_TRACKING_USERNAME"] = (
    "yahiaehab10"  # Replace with your DagsHub username
)
os.environ["MLFLOW_TRACKING_PASSWORD"] = (
    "b1e53999e80aecdcf2b9a9e6d0e2fea6ae02fa4c"  # Replace with your DagsHub token
)

# Set the correct MLflow tracking URI for DagsHub
mlflow.set_tracking_uri(
    "https://dagshub.com/yahiaehab10/end-to-end-sentiment-analysis.mlflow"
)

# Set the experiment name
mlflow.set_experiment("TF-IDF with (1,3) n-grams and max features=10000")

2025/09/20 13:50:26 INFO mlflow.tracking.fluent: Experiment with name 'TF-IDF with (1,3) n-grams and max features=10000' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/8effdb1ca2684b3386ea63bd86ffc0e5', creation_time=1758365426688, experiment_id='2', last_update_time=1758365426688, lifecycle_stage='active', name='TF-IDF with (1,3) n-grams and max features=10000', tags={}>

## Experiment

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os

import mlflow.sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [3]:
df = pd.read_csv("processed_reddit_data.csv").dropna(subset=["clean_comment"])
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [4]:
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 3)

    # Vectorization
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X = vectorizer.fit_transform(df["clean_comment"]).toarray()
    y = df["sentiment"].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    with mlflow.start_run():
        # Set tags
        mlflow.set_tag("mlflow.runName", f"TF-IDF (1,3) max_features={max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add description
        mlflow.set_tag("description", f"Experimenting TF-IDF with (1,3) n-grams and max features={max_features}")

        # Log parameters
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("max_features", max_features)
        mlflow.log_param("vectorizer_type", "TfidfVectorizer")

        # Log model parameters
        n_estimators = 100
        max_depth = 20
        random_state = 42

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("random_state", random_state)

        # Model training
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if label not in ["accuracy", "macro avg", "weighted avg"]:
                mlflow.log_metric(f"{label}_precision", metrics["precision"])
                mlflow.log_metric(f"{label}_recall", metrics["recall"])
                mlflow.log_metric(f"{label}_f1-score", metrics["f1-score"])

        # Log confusion matrix as an artifact
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(y), yticklabels=np.unique(y))
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        cm_path = "confusion_matrix.png"
        plt.savefig(cm_path)
        plt.close()
        mlflow.log_artifact(cm_path)
        os.remove(cm_path)

        # Log the model
        mlflow.sklearn.log_model(model, "random_forest_model")

max_features_values = [5000, 10000, 20000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

KeyError: 'sentiment'